# 입문자를 위한, 파이썬/R 데이터 분석      

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 > [법정동코드전체자료] )

]

### 1. Import Packages

In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [37]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [38]:
my_key

'oKp2k%2FHFgJt4W12ji1GMWWj0hDj8yQBH6IX3OBtt%2F2gHg0dBYlZpHZQTJLzkrtbdLsDREVVvIIx5Bp4KT5PdLg%3D%3D'

#### : Pre-defined

In [39]:
#"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [40]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [41]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202404"
svc_key = "serviceKey=" + my_key

In [42]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202404&serviceKey=oKp2k%2FHFgJt4W12ji1GMWWj0hDj8yQBH6IX3OBtt%2F2gHg0dBYlZpHZQTJLzkrtbdLsDREVVvIIx5Bp4KT5PdLg%3D%3D'

#### : Request Test

In [43]:
response = requests.get(url)
response

<Response [200]>

In [44]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     245,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼푸르지오</아파트><월>4</월><일>2</일><전용면적>84.9097</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754-1</지번><지역코드>11680</지역코드><층>16</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     196,000</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>4</월><일>5</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층>22</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     268,000</거래금액><거래유형>중개거래</거래유형><건축년도>2012</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>개나리SK뷰</아파트><월>4</월><일>5</일><전용면적>84.99</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>716-3</지번><지역코드>11680</지역코드><

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [45]:
lawd_cd = pd.read_csv("data/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [46]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
lawd_cd.columns

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')

In [47]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [48]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49850 entries, 0 to 49849
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    49850 non-null  int64 
 1   name    49850 non-null  object
 2   check   49850 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [49]:
in_name = "강남구"

In [50]:
lawd_cd['name'].str.contains(in_name)

0        False
1        False
2        False
3        False
4        False
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Name: name, Length: 49850, dtype: bool

In [51]:
lawd_cd['check'] == "존재"

0        True
1        True
2        True
3        True
4        True
         ... 
49845    True
49846    True
49847    True
49848    True
49849    True
Name: check, Length: 49850, dtype: bool

In [52]:
(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")

0        False
1        False
2        False
3        False
4        False
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Length: 49850, dtype: bool

In [53]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]

,code,name,check
1003,1168000000,서울특별시 강남구,존재
1037,1168010100,서울특별시 강남구 역삼동,존재
1039,1168010300,서울특별시 강남구 개포동,존재
1040,1168010400,서울특별시 강남구 청담동,존재
1041,1168010500,서울특별시 강남구 삼성동,존재
1042,1168010600,서울특별시 강남구 대치동,존재
1043,1168010700,서울특별시 강남구 신사동,존재
1044,1168010800,서울특별시 강남구 논현동,존재
1045,1168011000,서울특별시 강남구 압구정동,존재
1046,1168011100,서울특별시 강남구 세곡동,존재


In [54]:
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp

,code,name,check
1003,1168000000,서울특별시 강남구,존재


In [55]:
tmp.loc[1003,'code']    # 가져오기는 하나 문제

1168000000

In [56]:
tmp.iloc[0,0]           # 인덱스로 데이터를 가져오도록

1168000000

In [57]:
tmp = tmp.iloc[0,0]
print(tmp)
type(tmp)

1168000000


numpy.int64

In [58]:
tmp

1168000000

In [59]:
str(tmp)

'1168000000'

In [60]:
tmp.astype(str)

'1168000000'

In [61]:
tmp = tmp.astype(str)
tmp

'1168000000'

In [62]:
tmp[0:5]

'11680'

In [63]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11680'

In [64]:
# Function 작성 :
def find_lawd_cd(in_text):
    result = lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    
    return result

In [65]:
in_name = "강남구"
find_lawd_cd(in_name)

'11680'

In [66]:
find_lawd_cd('송파구')

'11710'

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [79]:
in_lawd = input("검색지역 입력 : ")

검색지역 입력 : 송파구


In [80]:
in_lawd

'송파구'

In [81]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 202010


In [82]:
in_deal_ymd

'202010'

In [83]:
in_svckey = my_key

In [84]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'11710'

In [85]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [86]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11710&DEAL_YMD=202010&serviceKey=oKp2k%2FHFgJt4W12ji1GMWWj0hDj8yQBH6IX3OBtt%2F2gHg0dBYlZpHZQTJLzkrtbdLsDREVVvIIx5Bp4KT5PdLg%3D%3D'

In [87]:
response = requests.get(url)
response

<Response [200]>

In [88]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     127,000</거래금액><거래유형> </거래유형><건축년도>1993</건축년도><년>2020</년><동> </동><등기일자> </등기일자><매도자> </매도자><매수자> </매수자><법정동> 잠실동</법정동><아파트>신동아예지움</아파트><월>10</월><일>6</일><전용면적>132</전용면적><중개사소재지> </중개사소재지><지번>307-2</지번><지역코드>11710</지역코드><층>10</층><해제사유발생일>20.11.20</해제사유발생일><해제여부>O</해제여부></item><item><거래금액>     222,800</거래금액><거래유형> </거래유형><건축년도>1978</건축년도><년>2020</년><동> </동><등기일자> </등기일자><매도자> </매도자><매수자> </매수자><법정동> 잠실동</법정동><아파트>주공아파트 5단지</아파트><월>10</월><일>7</일><전용면적>76.5</전용면적><중개사소재지> </중개사소재지><지번>27</지번><지역코드>11710</지역코드><층>7</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     216,000</거래금액><거래유형> </거래유형><건축년도>1978</건축년도><년>2020</년><동> </동><등기일자> </등기일자><매도자> </매도자><매수자> </매수자><법정동> 잠실동</법정동><아파트>주공아파트 5단지</아파트><월>10</월><일>14</일><전용면적>76.5</전용면적><중개사소재지> </중개사소재지><지번>27</지번><지역코드>11710</지역코드><층>12</층><해제사유발생일> </해제사유발생일

### 3. Data Preprocessng 

#### : web data / API output

In [89]:
#### BS 시도 : 잘안됨
soup = BeautifulSoup(response.text, "lxml")
soup

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item>&lt;거래금액&gt;     127,000거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1993건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자&gt;&lt;매수자&gt; 매수자&gt;&lt;법정동&gt; 잠실동법정동&gt;&lt;아파트&gt;신동아예지움아파트&gt;&lt;월&gt;10월&gt;&lt;일&gt;6일&gt;&lt;전용면적&gt;132전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;307-2지번&gt;&lt;지역코드&gt;11710지역코드&gt;&lt;층&gt;10층&gt;&lt;해제사유발생일&gt;20.11.20해제사유발생일&gt;&lt;해제여부&gt;O해제여부&gt;</item><item>&lt;거래금액&gt;     222,800거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1978건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자&gt;&lt;매수자&gt; 매수자&gt;&lt;법정동&gt; 잠실동법정동&gt;&lt;아파트&gt;주공아파트 5단지아파트&gt;&lt;월&gt;10월&gt;&lt;일&gt;7일&gt;&lt;전용면적&gt;76.5전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;27지번&gt;&lt;지역코드&gt;11710지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;

In [90]:
soup.find_all('item')

[<item>&lt;거래금액&gt;     127,000거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1993건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자&gt;&lt;매수자&gt; 매수자&gt;&lt;법정동&gt; 잠실동법정동&gt;&lt;아파트&gt;신동아예지움아파트&gt;&lt;월&gt;10월&gt;&lt;일&gt;6일&gt;&lt;전용면적&gt;132전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;307-2지번&gt;&lt;지역코드&gt;11710지역코드&gt;&lt;층&gt;10층&gt;&lt;해제사유발생일&gt;20.11.20해제사유발생일&gt;&lt;해제여부&gt;O해제여부&gt;</item>,
 <item>&lt;거래금액&gt;     222,800거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1978건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자&gt;&lt;매수자&gt; 매수자&gt;&lt;법정동&gt; 잠실동법정동&gt;&lt;아파트&gt;주공아파트 5단지아파트&gt;&lt;월&gt;10월&gt;&lt;일&gt;7일&gt;&lt;전용면적&gt;76.5전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;27지번&gt;&lt;지역코드&gt;11710지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>,
 <item>&lt;거래금액&gt;     216,000거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1978건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자

In [91]:
item = soup.find('item')
item

<item>&lt;거래금액&gt;     127,000거래금액&gt;&lt;거래유형&gt; 거래유형&gt;&lt;건축년도&gt;1993건축년도&gt;&lt;년&gt;2020년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt; 매도자&gt;&lt;매수자&gt; 매수자&gt;&lt;법정동&gt; 잠실동법정동&gt;&lt;아파트&gt;신동아예지움아파트&gt;&lt;월&gt;10월&gt;&lt;일&gt;6일&gt;&lt;전용면적&gt;132전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;307-2지번&gt;&lt;지역코드&gt;11710지역코드&gt;&lt;층&gt;10층&gt;&lt;해제사유발생일&gt;20.11.20해제사유발생일&gt;&lt;해제여부&gt;O해제여부&gt;</item>

In [92]:
#item.find('거래금액')#.text

In [93]:
#### XML 파서로 시도
import xml.etree.ElementTree as ET

In [94]:
response#.content#.text

<Response [200]>

In [95]:
root = ET.fromstring(response.content)
root

<Element 'response' at 0x115460d10>

In [96]:
item_list = []

In [97]:
root.find('body').find('items')[0]

<Element 'item' at 0x115460ef0>

In [98]:
type(root.find('body').find('items'))

xml.etree.ElementTree.Element

In [99]:
child = root.find('body').find('items')[0]
child

<Element 'item' at 0x115460ef0>

In [100]:
child.findall('*')#[0]

[<Element '거래금액' at 0x115460f40>,
 <Element '거래유형' at 0x115460f90>,
 <Element '건축년도' at 0x115464040>,
 <Element '년' at 0x1154640e0>,
 <Element '동' at 0x115464180>,
 <Element '등기일자' at 0x1154641d0>,
 <Element '매도자' at 0x115464220>,
 <Element '매수자' at 0x115464270>,
 <Element '법정동' at 0x1154642c0>,
 <Element '아파트' at 0x115464310>,
 <Element '월' at 0x1154643b0>,
 <Element '일' at 0x115464450>,
 <Element '전용면적' at 0x1154644a0>,
 <Element '중개사소재지' at 0x1154644f0>,
 <Element '지번' at 0x115464540>,
 <Element '지역코드' at 0x115464590>,
 <Element '층' at 0x115464630>,
 <Element '해제사유발생일' at 0x115464680>,
 <Element '해제여부' at 0x1154646d0>]

In [101]:
element = child.findall('*')[0]

In [102]:
element.tag.strip()

'거래금액'

In [103]:
element.text.strip()

'127,000'

In [104]:
data = {}

In [105]:
tag = element.tag.strip()
text = element.text.strip()
data[tag] = text

In [106]:
data

{'거래금액': '127,000'}

In [107]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [108]:
items_list = find_xml_items(response)
items_list

[{'거래금액': '127,000',
  '거래유형': '',
  '건축년도': '1993',
  '년': '2020',
  '동': '',
  '등기일자': '',
  '매도자': '',
  '매수자': '',
  '법정동': '잠실동',
  '아파트': '신동아예지움',
  '월': '10',
  '일': '6',
  '전용면적': '132',
  '중개사소재지': '',
  '지번': '307-2',
  '지역코드': '11710',
  '층': '10',
  '해제사유발생일': '20.11.20',
  '해제여부': 'O'},
 {'거래금액': '222,800',
  '거래유형': '',
  '건축년도': '1978',
  '년': '2020',
  '동': '',
  '등기일자': '',
  '매도자': '',
  '매수자': '',
  '법정동': '잠실동',
  '아파트': '주공아파트 5단지',
  '월': '10',
  '일': '7',
  '전용면적': '76.5',
  '중개사소재지': '',
  '지번': '27',
  '지역코드': '11710',
  '층': '7',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '216,000',
  '거래유형': '',
  '건축년도': '1978',
  '년': '2020',
  '동': '',
  '등기일자': '',
  '매도자': '',
  '매수자': '',
  '법정동': '잠실동',
  '아파트': '주공아파트 5단지',
  '월': '10',
  '일': '14',
  '전용면적': '76.5',
  '중개사소재지': '',
  '지번': '27',
  '지역코드': '11710',
  '층': '12',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '235,000',
  '거래유형': '',
  '건축년도': '1981',
  '년': '2020',
  '동': '',
  '등기일자': '',
  '매도자': '',
  '매

### 4. Result : Display & Save as file 

In [109]:
result = pd.DataFrame(items_list)

In [110]:
result

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"127,000",,1993,2020,,,,,잠실동,신동아예지움,10,6,132,,307-2,11710,10,20.11.20,O
1,"222,800",,1978,2020,,,,,잠실동,주공아파트 5단지,10,7,76.5,,27,11710,7,,
2,"216,000",,1978,2020,,,,,잠실동,주공아파트 5단지,10,14,76.5,,27,11710,12,,
3,"235,000",,1981,2020,,,,,잠실동,우성아파트,10,17,131.08,,101-1,11710,2,,
4,"222,500",,2008,2020,,,,,잠실동,리센츠,10,17,84.99,,22,11710,12,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,"114,700",,2011,2020,,,,,마천동,송파파크데일2단지,10,14,84.13,,607,11710,6,,
233,"53,000",,1998,2020,,,,,마천동,한보1,10,23,59.26,,175,11710,5,,
234,"88,000",,2011,2020,,,,,마천동,송파파크데일1단지,10,27,59.91,,582,11710,5,,
235,"90,500",,2011,2020,,,,,마천동,송파파크데일1단지,10,31,59.91,,582,11710,10,21.01.28,O


In [111]:
result.shape

(237, 19)

In [112]:
result.head()

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"127,000",,1993,2020,,,,,잠실동,신동아예지움,10,6,132,,307-2,11710,10,20.11.20,O
1,"222,800",,1978,2020,,,,,잠실동,주공아파트 5단지,10,7,76.5,,27,11710,7,,
2,"216,000",,1978,2020,,,,,잠실동,주공아파트 5단지,10,14,76.5,,27,11710,12,,
3,"235,000",,1981,2020,,,,,잠실동,우성아파트,10,17,131.08,,101-1,11710,2,,
4,"222,500",,2008,2020,,,,,잠실동,리센츠,10,17,84.99,,22,11710,12,,


In [113]:
result[result['아파트'] == '래미안파크팰리스']

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
92,"174,500",,2007,2020,,,,,가락동,래미안파크팰리스,10,16,114.756,,165,11710,13,,


#### : file name

In [114]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

'송파구_202010'

In [115]:
file_name = f"data/result_{name_info}.csv"
file_name

'data/result_송파구_202010.csv'

In [116]:
result.to_csv(file_name, index=False, encoding='cp949')

## Appendix : Function 

In [117]:
def deal_by_lawd(cd_name, ymd):
    
    func_cd = find_lawd_cd(cd_name)
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + func_cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = cd_name + "_" + ymd
#     file_name = f"data/result_{name_info}.csv"

#     result.to_csv(file_name, index=False, encoding='cp949')

    
    
    return result

In [118]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
func_gu = input("검색지역 입력 : ")
print(func_gu)

func_ymd = input("계약월 입력(예: 201911) : ")
print(func_ymd)

검색지역 입력 : 송파구
송파구
계약월 입력(예: 201911) : 202010
202010


In [119]:
deal_by_lawd(func_gu, func_ymd)

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"127,000",,1993,2020,,,,,잠실동,신동아예지움,10,6,132,,307-2,11710,10,20.11.20,O
1,"222,800",,1978,2020,,,,,잠실동,주공아파트 5단지,10,7,76.5,,27,11710,7,,
2,"216,000",,1978,2020,,,,,잠실동,주공아파트 5단지,10,14,76.5,,27,11710,12,,
3,"235,000",,1981,2020,,,,,잠실동,우성아파트,10,17,131.08,,101-1,11710,2,,
4,"222,500",,2008,2020,,,,,잠실동,리센츠,10,17,84.99,,22,11710,12,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,"114,700",,2011,2020,,,,,마천동,송파파크데일2단지,10,14,84.13,,607,11710,6,,
233,"53,000",,1998,2020,,,,,마천동,한보1,10,23,59.26,,175,11710,5,,
234,"88,000",,2011,2020,,,,,마천동,송파파크데일1단지,10,27,59.91,,582,11710,5,,
235,"90,500",,2011,2020,,,,,마천동,송파파크데일1단지,10,31,59.91,,582,11710,10,,


In [120]:
test = result[result['법정동'] == '가락동']
test['거래금액'] = test['거래금액'].str.replace(',', '').astype(int)
test

/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_1268/8260136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['거래금액'] = test['거래금액'].str.replace(',', '').astype(int)


,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
83,114500,,1997,2020,,,,,가락동,가락금호아파트,10,9,59.91,,95-1,11710,5,,
84,134000,,1997,2020,,,,,가락동,가락금호아파트,10,10,85,,95-1,11710,17,,
85,105000,,2006,2020,,,,,가락동,성원상떼빌,10,11,128.71,,80,11710,10,,
86,75000,,1998,2020,,,,,가락동,동부썬빌,10,11,142.388,,99-6,11710,15,,
87,97500,,1991,2020,,,,,가락동,현대7,10,12,84.65,,171-5,11710,11,,
88,98500,,2003,2020,,,,,가락동,sk파크타워,10,12,108.82,,166-2,11710,17,,
89,64000,,2004,2020,,,,,가락동,동일씨티빌,10,13,84.84,,16-12,11710,4,,
90,115500,,1997,2020,,,,,가락동,가락금호아파트,10,14,59.91,,95-1,11710,5,,
91,144500,,1997,2020,,,,,가락동,가락금호아파트,10,15,85,,95-1,11710,13,,
92,174500,,2007,2020,,,,,가락동,래미안파크팰리스,10,16,114.756,,165,11710,13,,


In [121]:
test['거래금액'].mean()

110827.58620689655

### 추가 실습 :
- 공공API 데이터 끌어오기 친숙해지기
- **(필수) 특정월에 서울시 전체 구의 데이터를 한번에 가져오기(입력값: 연월 202402)**
- (선택) 특정 서울시 하나의 구의 과거 1년치 데이터를 한번에 가져오기(입력값: 특정구 하나) 

- 기상청_중기예보 조회서비스 (추가 API 사용신청 필요)    
https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059468

In [168]:
import requests

url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidFcst'
params ={'serviceKey' : my_key, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 'stnId' : '108', 'tmFc' : '202406130200' }

response = requests.get(url, params=params)
#print(response.content)

In [169]:
print(response.text)

<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>


### 코드리뷰 과제(1)

- 서울시 모든 구의 리스트 추려내는 방법

In [124]:
lawd_cd.head()

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


- 파이썬으로 구하는 방법

In [125]:
df = lawd_cd
tmp = df[df['name'].str.contains('서울특별시')]

In [126]:
#list(tmp['name'])

In [127]:
##### 코드리뷰 대상입니다 ####
name_list = list(tmp['name'])
goo_name = []
for name in name_list:
    if name[-1] == "구":
        name_split = name.split(' ')
        goo_name.append(name_split[1])
print(goo_name)
len(goo_name)

['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']


25

- 판다스로 구하는 방법

In [128]:
df = lawd_cd

In [129]:
#df[df['name'].str.contains('서울특별시')]['name'].str.split().str[1].dropna().unique()

In [130]:
##### 코드리뷰 대상입니다 ####
seoul_gu_list = df[df['name'].str.contains('서울특별시')]['name'].str.split().str[1].dropna().unique()
seoul_gu_list

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [131]:
seoul_gu_list = list(set(seoul_gu_list))
len(seoul_gu_list)

25

- **(필수) 특정월에 서울시 전체 구의 데이터를 한번에 가져오기(입력값: 연월 202402)**

In [137]:
seoul_gu_list.sort()

In [139]:
seoul_gu_list

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [134]:
func_ymd = '202403'

In [135]:
#deal_by_lawd(seoul_gu_list[5], func_ymd)

In [136]:
for gu in seoul_gu_list:
    print(gu)
    print(deal_by_lawd(gu, func_ymd))

마포구
        거래금액  거래유형  건축년도     년    동      등기일자 매도자 매수자  법정동           아파트  월  \
0    138,100  중개거래  2014  2024  204  24.05.28  개인  개인  아현동  마포래미안푸르지오2단지  3   
1    142,800  중개거래  2014  2024  307  24.04.30  개인  개인  아현동  마포래미안푸르지오3단지  3   
2    101,000  중개거래  2003  2024                 개인  개인  아현동        삼환나띠르빌  3   
3    124,700  중개거래  2008  2024                 개인  개인  아현동     마포트라팰리스2차  3   
4    135,000  중개거래  2014  2024  305  24.04.29  개인  개인  아현동  마포래미안푸르지오3단지  3   
..       ...   ...   ...   ...  ...       ...  ..  ..  ...           ... ..   
203   85,500  중개거래  2003  2024  202  24.05.20  개인  개인  상암동    상암월드컵파크2단지  3   
204  114,750  중개거래  2006  2024                 개인  개인  상암동    상암월드컵파크4단지  3   
205  100,000  중개거래  2003  2024                 개인  개인  상암동    상암월드컵파크3단지  3   
206   90,000  중개거래  2003  2024                 개인  개인  상암동    상암월드컵파크2단지  3   
207  124,000  중개거래  2006  2024                 개인  개인  상암동    상암월드컵파크4단지  3   

      일     전용면적          중개사소재지    지번   지역코드  

       거래금액  거래유형  건축년도     년    동      등기일자 매도자 매수자  법정동        아파트  월   일  \
0    61,000  중개거래  1986  2024   12  24.05.21  개인  개인  월계동         미성  3   1   
1    77,300  중개거래  2006  2024  113  24.05.10  개인  개인  월계동     롯데캐슬루나  3   1   
2    83,000  중개거래  2002  2024                 개인  개인  월계동    한진한화그랑빌  3   1   
3    74,000  중개거래  2020  2024  101  24.06.07  개인  개인  월계동  월계센트럴아이파크  3   1   
4    85,800  중개거래  2020  2024  107  24.05.27  개인  개인  월계동  월계센트럴아이파크  3   2   
..      ...   ...   ...   ...  ...       ...  ..  ..  ...        ... ..  ..   
296  51,500  중개거래  1991  2024                 개인  개인  중계동      중계무지개  3  30   
297  93,500  중개거래  1992  2024                 개인  개인  중계동        주공5  3  30   
298  79,200  중개거래  2005  2024                 개인  개인  중계동     한화꿈에그린  3  30   
299  60,000  중개거래  1998  2024                 개인  개인  중계동       현대그린  3  30   
300  51,700  중개거래  1990  2024  114  24.05.17  개인  개인  중계동    중계그린1단지  3  30   

        전용면적          중개사소재지     지번   지역코드   층 해제사유

        거래금액  거래유형  건축년도     년    동      등기일자 매도자 매수자  법정동              아파트  \
0    201,000  중개거래  2004  2024  301  24.05.30  개인  개인  방배동        방배3차현대홈타운   
1    118,000  중개거래  2003  2024  101  24.04.26  개인  개인  방배동  삼환나띠르빌(1002-22)   
2    137,000  중개거래  2002  2024                 개인  개인  방배동        방배2차상지리츠빌   
3    141,500  중개거래  1991  2024  108  24.05.03  개인  개인  방배동             방배우성   
4    149,000  중개거래  1975  2024    2  24.03.28  개인  개인  방배동          삼호1동~3동   
..       ...   ...   ...   ...  ...       ...  ..  ..  ...              ...   
200  335,000  중개거래  2021  2024                 개인  개인  서초동           서초그랑자이   
201  261,000  중개거래  2018  2024                 개인  개인  서초동       래미안서초에스티지S   
202  144,000  중개거래  2020  2024                 개인  개인  서초동       서초센트럴IPARK   
203  142,500  중개거래  2014  2024  101  24.05.10  개인  개인  내곡동           서초더샵포레   
204  143,000  중개거래  2015  2024  203  24.06.05  개인  개인  내곡동        서초포레스타2단지   

     월   일     전용면적  중개사소재지       지번   지역코드   층 해제사

       거래금액  거래유형  건축년도     년     동      등기일자 매도자 매수자  법정동         아파트  월   일  \
0    54,000  중개거래  1998  2024  209동  24.06.05  개인  개인  돈암동   한진(609-1)  3   1   
1    72,000  중개거래  2006  2024   103  24.06.03  개인  개인  돈암동    돈암동더샵아파트  3   1   
2    43,000  중개거래  1991  2024   101  24.03.15  개인  개인  돈암동     돈암현대아파트  3   3   
3    57,500  중개거래  1998  2024   107  24.03.22  개인  개인  돈암동          한신  3   3   
4    61,000  중개거래  2005  2024   106  24.03.11  개인  개인  돈암동  일신건영휴먼빌아파트  3   4   
..      ...   ...   ...   ...   ...       ...  ..  ..  ...         ... ..  ..   
211  66,000  중개거래  1998  2024   125  24.05.24  개인  개인  석관동          두산  3  27   
212  76,500  중개거래  1998  2024   107  24.05.24  개인  개인  석관동          두산  3  27   
213  84,000  중개거래  2019  2024                  개인  개인  석관동     래미안아트리치  3  28   
214  74,500  중개거래  1998  2024                  개인  개인  석관동          두산  3  29   
215  30,800   직거래  1997  2024   101  24.04.30  개인  개인  석관동          삼성  3  29   

       전용면적          중개사소재지

       거래금액  거래유형  건축년도     년           동      등기일자 매도자 매수자    법정동  \
0   209,000  중개거래  2011  2024         101  24.05.02  개인  개인  회현동1가   
1    67,000  중개거래  1970  2024              24.04.26  개인  개인  충무로4가   
2    24,000  중개거래  2018  2024  충무로 헤센 스마트  24.04.17  개인  개인  충무로5가   
3    25,400  중개거래  2018  2024                        개인  개인  충무로5가   
4    47,000  중개거래  2018  2024  충무로 헤센 스마트  24.03.29  개인  개인  충무로5가   
..      ...   ...   ...   ...         ...       ...  ..  ..    ...   
56  128,000  중개거래  2017  2024                        개인  개인  만리동2가   
57  125,000  중개거래  2017  2024                        개인  개인  만리동2가   
58  128,000  중개거래  2017  2024                        개인  개인  만리동2가   
59  155,000  중개거래  2017  2024         110  24.05.31  개인  개인  만리동2가   
60  135,000  중개거래  2017  2024                        개인  개인  만리동2가   

           아파트  월   일     전용면적         중개사소재지     지번   지역코드   층   해제사유발생일 해제여부  
0   남산롯데캐슬아이리스  3  12   133.98          서울 중구    208  11140  24               

        거래금액  거래유형  건축년도     년    동      등기일자 매도자 매수자    법정동          아파트  월  \
0     54,500  중개거래  2004  2024       24.05.17  개인  개인  충정로3가        충정리시온  3   
1    100,000  중개거래  2011  2024  213  24.05.24  개인  개인    냉천동      돈의문센트레빌  3   
2     76,000  중개거래  2006  2024                 개인  개인    천연동        천연뜨란채  3   
3    113,800  중개거래  1995  2024                 개인  개인    영천동        독립문삼호  3   
4    108,000  중개거래  1995  2024                 개인  개인    영천동        독립문삼호  3   
..       ...   ...   ...   ...  ...       ...  ..  ..    ...          ... ..   
130  119,000  중개거래  2015  2024                 개인  개인   남가좌동  DMC파크뷰자이2단지  3   
131  126,000  중개거래  2015  2024                 개인  개인   남가좌동  DMC파크뷰자이1단지  3   
132   83,000  중개거래  1999  2024                 개인  개인   남가좌동       남가좌동현대  3   
133   88,000  중개거래  2005  2024                 개인  개인   남가좌동     래미안남가좌2차  3   
134   80,000  중개거래  2018  2024  107  24.05.03  개인  개인   남가좌동   DMC센트럴아이파크  3   

      일    전용면적     중개사소재지   지번   지역코드 